https://mp.weixin.qq.com/s/k0H6xFXOZDqLVvWouW9GAQ

In [26]:
class A():
    def __init__(self,data):
        self.data = data
        self.a = 0
        self.b = 0
        self.c = 0
    def aaa(self):
        self.a = 10+self.data
        return self.a
    def bbb(self):
        self.b= self.a+self.a
        return self.b
    def ccc(self):
        self.c = self.a+self.b
        return self.c

In [33]:
A(1).aaa()
A(1).bbb()

0

In [15]:
class B():
    def __init__(self,data):
        self.data = data

    def aaa(self):
        a = 10+self.data
        return
    def bbb(self, a):
        b= a + 100
        return b

In [16]:
B(100).bbb()

TypeError: bbb() missing 1 required positional argument: 'a'

In [1]:
import os
import exifread
from decimal import Decimal
from position_utils import *
import requests
import json
import datetime

In [23]:
class Location(object):

    def __init__(self, image_path):
        self.img_path = image_path
        self.api_key = "9d3c1152e2c9b7d2e323d054dc0c52a6"
        self.url_get_position = 'https://restapi.amap.com/v3/geocode/regeo?key={}&location={}'

    def run(self):
        data_time = self.__formed_time()
        coordinate = self.__get_image_ability()
        print(f'获取到经度、纬度是:{coordinate}')

        if not coordinate:
            return

        address = self.__get_address(coordinate)# 根据经纬度，获取详细地址
        # 检验坐标值
        # https://lbs.amap.com/console/show/picker
        print(f'照片于{data_time}拍摄于{address}')

    def __get_address(self, location):#对经度和纬度数据做处理，保留6位小数:param data: 原始经度和纬度值:return:

        resp = requests.get(self.url_get_position.format(self.api_key, location))
        location_data = json.loads(resp.text)
        address = location_data.get('regeocode').get('formatted_address')

        return address

    def __format_lati_long_data(self, data):#对经度和纬度数据做处理，保留6位小数:param data: 原始经度和纬度值:return:

        data_list_tmp = str(data).replace('[', '').replace(']', '').split(',')# 删除左右括号和空格
        data_list = [data.strip() for data in data_list_tmp]

        data_tmp = data_list[-1].split('/')# 替换秒的值
        data_sec = int(data_tmp[0]) / int(data_tmp[1]) / 3600# 秒的值
        data_tmp = data_list[-2]# 替换分的值
        data_minute = int(data_tmp) / 60# 分的值
        data_degree = int(data_list[0])# 度的值

        result = "%.6f" % (data_degree + data_minute + data_sec)# 高德API只能识别小数点后6位， 需转为浮点数并保留6位小数
        return float(result)

    def __get_image_ability(self):#获取图片属性值，包含：经纬度、拍摄时间等:param picture_name::return:

        img_exif = exifread.process_file(open(self.img_path, 'rb'))# 利用exifread库读取图片属性

        if img_exif:#如果能够提取数据
            
            latitude_gps = img_exif['GPS GPSLatitude']# 纬度数
            latitude_direction = img_exif['GPS GPSLatitudeRef']# N,S 南北纬方向
            longitude_gps = img_exif['GPS GPSLongitude']# 经度数
            longitude_direction = img_exif['GPS GPSLongitudeRef']# E,W 东西经方向
            take_time = img_exif['EXIF DateTimeOriginal']# 拍摄时间
            format_time = str(take_time).split(" ")[0].replace(":", "-")# 拍摄时间标准格式

            if latitude_gps and longitude_gps and take_time:# 纬度、经度、拍摄时间
                latitude = self.__format_lati_long_data(latitude_gps)# 对纬度、经度值原始值作进一步的处理
                longitude = self.__format_lati_long_data(longitude_gps)# 对纬度、经度值原始值作进一步的处理
                # print(f'{longitude},{latitude}')
                location = wgs84togcj02(longitude, latitude)# 注意：gps坐标在高德地图逆编码不够精确，需转为火星坐标系
                return f'{location[0]},{location[1]}'
            
            else:
                print(f'获取的图片数据属性不完整')
                return ''
        else:
            print('抱歉，图片不是原图，没法获取到图片属性。')
            return ''

    def __formed_time(self):
            img_exif = exifread.process_file(open(self.img_path, 'rb'))# 利用exifread库读取图片属性
            take_time = img_exif['EXIF DateTimeOriginal']# 拍摄时间
            format_time = str(take_time).split(" ")[0].replace(":", "-")# 拍摄时间标准格式
            return format_time
        
    #def judge_time_met(self, take_time):#通知拍摄时间判断女朋友是否撒谎:param take_time::return:
        #format_time = str(take_time).split(" ")[0].replace(":", "-")# 拍摄时间
        #today = str(datetime.date.today())# 当天日期

        #if format_time == today:
            #return False
        #else:
            #return True

In [73]:
if __name__ == '__main__':
    # 女朋友发过来的图片【原图】
    location = Location(r'D:\321.jpg')

    # 找到女朋友的地理位置
    location.run()

获取到经度、纬度是:118.24003381322925,25.494841366678727
照片于2019-08-26拍摄于福建省泉州市德化县浔中镇浔北路8德化第一中学


In [119]:
if __name__ == '__main__':
    # 女朋友发过来的图片【原图】
    location = Location(r'D:\123.jpg')

    # 找到女朋友的地理位置
    location.run()

获取到经度、纬度是:118.24279262103555,25.487639137166987
陈晨当前位置在:福建省泉州市德化县龙浔镇隆中路12幢金龙·中心城(建设中)


In [70]:
if __name__ == '__main__':
    # 女朋友发过来的图片【原图】
    location = Location(r'D:\555.jpg')

    # 找到女朋友的地理位置
    location.run()

KeyError: 'EXIF DateTimeOriginal'

In [82]:
img_exif = exifread.process_file(open(r'D:\aaa.jpg', 'rb'))# 利用exifread库读取图片属性
#img_exif = exifread.process_file(open(r'D:\876.jpg', 'rb'))# 利用exifread库读取图片属性


#latitude_gps = img_exif['GPS GPSLatitude']# 纬度数
#latitude_direction = img_exif['GPS GPSLatitudeRef']# N,S 南北纬方向
#longitude_gps = img_exif['GPS GPSLongitude']# 经度数
#longitude_direction = img_exif['GPS GPSLongitudeRef']# E,W 东西经方向
take_time = img_exif['EXIF DateTimeOriginal']# 拍摄时间
format_time = str(take_time).split(" ")[0].replace(":", "-")# 拍摄时间标准格式
#img_exif

{'Image Make': (0x010F) ASCII=Apple @ 134,
 'Image Model': (0x0110) ASCII=iPhone 6s @ 140,
 'Image Orientation': (0x0112) Short=Rotated 90 CW @ 42,
 'Image XResolution': (0x011A) Ratio=72 @ 150,
 'Image YResolution': (0x011B) Ratio=72 @ 158,
 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 78,
 'Image Software': (0x0131) ASCII=12.4.1 @ 166,
 'Image DateTime': (0x0132) ASCII=2019:09:10 12:58:44 @ 174,
 'Image YCbCrPositioning': (0x0213) Short=Centered @ 114,
 'Image ExifOffset': (0x8769) Long=194 @ 126,
 'Thumbnail Compression': (0x0103) Short=JPEG (old-style) @ 1690,
 'Thumbnail XResolution': (0x011A) Ratio=72 @ 1758,
 'Thumbnail YResolution': (0x011B) Ratio=72 @ 1766,
 'Thumbnail ResolutionUnit': (0x0128) Short=Pixels/Inch @ 1726,
 'Thumbnail JPEGInterchangeFormat': (0x0201) Long=1774 @ 1738,
 'Thumbnail JPEGInterchangeFormatLength': (0x0202) Long=13116 @ 1750,
 'EXIF ExposureTime': (0x829A) Ratio=1/33 @ 584,
 'EXIF FNumber': (0x829D) Ratio=11/5 @ 592,
 'EXIF ExposureProgram': (0

In [83]:
take_time

(0x9003) ASCII=2019:09:10 12:58:44 @ 600

In [ ]:
class Location(object):

    def __init__(self, image_path):
        self.img_path = image_path
        self.api_key = "9d3c1152e2c9b7d2e323d054dc0c52a6"
        self.url_get_position = 'https://restapi.amap.com/v3/geocode/regeo?key={}&location={}'

    def run(self):
        coordinate = self.__get_image_ability()
        print(f'获取到经度、纬度是:{coordinate}')

        if not coordinate:
            return

        address = self.__get_address(coordinate)# 根据经纬度，获取详细地址
        # 检验坐标值
        # https://lbs.amap.com/console/show/picker
        print(f'你女朋友当前位置在:{address}')

    def __get_address(self, location):#对经度和纬度数据做处理，保留6位小数:param data: 原始经度和纬度值:return:

        resp = requests.get(self.url_get_position.format(self.api_key, location))
        location_data = json.loads(resp.text)
        address = location_data.get('regeocode').get('formatted_address')

        return address

    def __format_lati_long_data(self, data):#对经度和纬度数据做处理，保留6位小数:param data: 原始经度和纬度值:return:



    def __get_image_ability(self):#获取图片属性值，包含：经纬度、拍摄时间等:param picture_name::return:

        img_exif = exifread.process_file(open(self.img_path, 'rb'))# 利用exifread库读取图片属性

        if img_exif:#如果能够提取数据
            
            latitude_gps = img_exif['GPS GPSLatitude']# 纬度数
            latitude_direction = img_exif['GPS GPSLatitudeRef']# N,S 南北纬方向
            longitude_gps = img_exif['GPS GPSLongitude']# 经度数
            longitude_direction = img_exif['GPS GPSLongitudeRef']# E,W 东西经方向
            take_time = img_exif['EXIF DateTimeOriginal']# 拍摄时间

            if latitude_gps and longitude_gps and take_time:# 纬度、经度、拍摄时间
                latitude = self.__format_lati_long_data(latitude_gps)# 对纬度、经度值原始值作进一步的处理
                longitude = self.__format_lati_long_data(longitude_gps)# 对纬度、经度值原始值作进一步的处理
                # print(f'{longitude},{latitude}')
                location = wgs84togcj02(longitude, latitude)# 注意：gps坐标在高德地图逆编码不够精确，需转为火星坐标系
                return f'{location[0]},{location[1]}'
            
            else:
                print(f'获取的图片数据属性不完整')
                return ''
        else:
            print('抱歉，图片不是原图，没法获取到图片属性。')
            return ''